#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Making a connection to a Cassandra instance my local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### 1. Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Return the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_query1 = "CREATE TABLE IF NOT EXISTS sessions" 
create_query1 =  create_query1 + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(create_query1)     
except Exception as e:
    print(e)

### Presentation:
For table <code>sessions</code>, the <code>session_id</code> is used as a partition key and <code>item_in_session</code> is used as clustering column to help make up a unique key and the queries will filter by these columns.

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sessions (session_id, item_in_session, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
## Adding in the SELECT statement to verify the data was entered into the table
query1 = "SELECT artist, song_title, song_length FROM sessions WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query1)
    x = PrettyTable()
    for row in rows:
        x.field_names = ['artist', 'song_title', 'song_length']
        x.add_row([row.artist, row.song_title, row.song_length])
    print(x)
except Exception as e:
    print(e)      

+-----------+---------------------------------+--------------------+
|   artist  |            song_title           |    song_length     |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


In [11]:
## Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_query2 = "CREATE TABLE IF NOT EXISTS users" 
create_query2 = create_query2 + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(create_query2)
except Exception as e:
    print(e)

### Presentation:
For table <code>users</code>, the <code>user_id</code> and <code>session_id</code> are used as composite partition key because the queries will filter by these columns and <code>item_in_session</code> is used as clustering column to help make up a unique key.

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])) 

In [13]:
query2 = "select artist_name, song_title, first_name, last_name from users WHERE session_id = 182 and user_id = 10"
try:    
    rows = session.execute(query2)
    y = PrettyTable()
    for row in rows:
        y.field_names = ['artist_name', 'song_title', 'first_name', 'last_name']
        y.add_row([row.artist_name, row.song_title, row.first_name, row.last_name])
    print(y)
except Exception as e:
    print(e)

+-------------------+------------------------------------------------------+------------+-----------+
|    artist_name    |                      song_title                      | first_name | last_name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


In [14]:
## Query 3: Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_query3 = "CREATE TABLE IF NOT EXISTS song_listens" 
create_query3 = create_query3 + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY(song_title, user_id))"
try:
    session.execute(create_query3)
except Exception as e:
    print(e)                

### Presentation:
For table <code>song_listens</code>, the <code>song_title</code> is used as a partition key and <code>user_id</code> is used as clustering column to help make up a unique key and the queries will filter by these columns.

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listens (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))  

In [16]:
query3 = "select first_name, last_name from song_listens WHERE song_title = 'All Hands Against His Own'"
try:    
    rows = session.execute(query3)
    z = PrettyTable()
    for row in rows:
        z.field_names = ['first_name', 'last_name']
        z.add_row([row.first_name, row.last_name])
    print(z)
except Exception as e:
    print(e)

+------------+-----------+
| first_name | last_name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [17]:
create_query1 = "DROP TABLE IF EXISTS sessions"
session.execute(create_query1)

dcreate_query2 = "DROP TABLE IF EXISTS users"
session.execute(create_query2)

create_query3 = "DROP TABLE IF EXISTS song_listens"
session.execute(create_query3)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()